In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as FS
from pyspark.sql import functions as f

In [0]:
spark = SparkSession.builder.appName("Indian Cricket Data").getOrCreate()

#### Cricket Data 

In [0]:
cricket_data = [("Virat Kohli", 85, 100, 75),
        ("Steve Smith", 90, 105, 80),
        ("Kane Williamson", 88, 95, 70)]
columns = ["Player", "Match1", "Match2", "Match3"]

In [0]:
cricket_df = spark.createDataFrame(data = cricket_data,schema = columns)
cricket_df.show()

**Pivote:** Conver Row to column format<br>
**UnPivote:** Convert Column to Row
**Convert the above Dataframe format into below format**<br>
```
+---------------+------+-----+
|         Player| Match|Score|
+---------------+------+-----+
|    Virat Kohli|Match1|   85|
|    Virat Kohli|Match2|  100|
|    Virat Kohli|Match3|   75|
|    Steve Smith|Match1|   90|
|    Steve Smith|Match2|  105|
|    Steve Smith|Match3|   80|
|Kane Williamson|Match1|   88|
|Kane Williamson|Match2|   95|
|Kane Williamson|Match3|   70|
+---------------+------+-----+

```

#### create unpivote data Frame

In [0]:
# stack_expr = "stack(1,'Match1', Match1) as (Match, Score)"
# stack_expr = "stack(2,'Match1', Match1,'Match2',Match2) as (Match, Score)"
stack_expr = "stack(3,'Match1', Match1,'Match2',Match2,'Match3',Match3) as (Match, Score)"
stack_df1 = cricket_df.select("Player",FS.expr(stack_expr))
stack_df1.show()

### Method-2

In [0]:
dataframe_match = (cricket_df.withColumn("matches",f.array(f.lit("Match1") ,f.lit("Match2") , f.lit("Match3") ))
       .withColumn("score_list",f.array(f.col("Match1") ,f.col("Match2") , f.col("Match3"))))


dataframe_match_zip = dataframe_match.withColumn("dict",f.arrays_zip("matches","score_list"))
dataframe_match_zip = dataframe_match_zip.drop("Match1","Match2","Match3","matches","score_list")
dataframe_match_zip.display()

In [0]:
dataframe_match_zip.withColumn("tmp",f.explode("dict")).withColumn("match",f.col("tmp.matches")).withColumn("score",f.col("tmp.score_list")).drop("dict","tmp","Match3").display()

### Pivote

In [0]:
## Pivote the data
pivote_data = stack_df1.groupBy("Player").pivot("Match").sum("Score")
pivote_data.show()

In [0]:
df = (
    stack_df1.withColumn("Match1",f.when(f.col("match") == "Match1",f.col("score"))
                        .when(f.col("match") == "Match2",f.col("score"))
                        .when(f.col("match") == "Match3",f.col("score"))
                        )
    
    .drop("match","score")
)
df.display()

In [0]:
emp_data = [
    ('Siva',1,30000),
    ('Ravi',2,40000),
    ('Prasad',1,50000),
    ('Arun',1,30000),
    ('Sai',2,20000)
]
emp_schema = "emp_name string , dep_id int , salary long"
df = spark.createDataFrame(data = emp_data , schema = emp_schema)
df.show()

In [0]:
df.groupBy("dep_id").agg(f.max("salary").alias("max_salary"),f.min("salary").alias("min_salary")).display()